In [ ]:
import os # To import test image files
import cv2 # To work with opencv images
from PIL import Image # Image submodule to work with pillow images
import pytesseract as pt # pytesseract module

In [ ]:
test_img_path = "../test_images/bill"
create_path = lambda f : os.path.join(test_img_path, f)

test_image_files = os.listdir(test_img_path)

for f in test_image_files:
    print(f)

In [ ]:
def show_image(img_path, size=(800, 800)):
    image = cv2.imread(img_path)
    image = cv2.resize(image, size)
    
    cv2.imshow("IMAGE", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [4]:
image_path = test_image_files[1] # 2, 3, 12, 1, 13, 15
path = create_path(image_path)

image = Image.open(path)
text = pt.image_to_string(image,config="--oem 3 --psm 6")

print(text)
show_image(path)

Use Image Preprocessing

In [ ]:
img = cv2.imread(path)

print("Image shape: ", path)

if img is not None:
    # Center the image
    img = cv2.copyMakeBorder(img, 50, 50, 50, 50, cv2.BORDER_CONSTANT, value=[255])

    # Up-sample (optional)
    img = cv2.resize(img, (0, 0), fx=2, fy=2)

    # Convert to gray-scale
    gry = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to create a binary image
    _, binary_img = cv2.threshold(gry, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # OCR
    txt = pt.image_to_string(gry, config="--psm 6")

    print(txt)
else:
    print("Failed to load the image.")

show_image(path)

In [ ]:
#identify the date
import re

match=re.findall(r'\d+[/.-]\d+[/.-]\d+', text)

st=" "
st=st.join(match)
print(st)

In [ ]:
import re


# Define regex pattern to match item lines
item_pattern = r'(.*?) [\d.]+'

# Extract items using regex
items = re.findall(item_pattern, text)

# Print the extracted items
print("Extracted Items:")
for item in items:
    print(item)



With Thrusholdings 

In [ ]:
img = cv2.imread(path)

print("Image shape: ", path)

if img is not None:
    # Center the image
    img = cv2.copyMakeBorder(img, 50, 50, 50, 50, cv2.BORDER_CONSTANT, value=[255])

    # Up-sample (optional)
    img = cv2.resize(img, (0, 0), fx=2, fy=2)

    # Convert to gray-scale
    gry = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to create a binary image
    _, binary_img = cv2.threshold(gry, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    

    # OCR
    txt = pt.image_to_string(binary_img, config="--psm 6")

    print(txt)
else:
    print("Failed to load the image.")

show_image(path)

 Thinning and Skeletonization


In [ ]:
import numpy as np

img = cv2.imread(path)

print("Image shape: ", path)

if img is not None:
    # Center the image
    img = cv2.copyMakeBorder(img, 50, 50, 50, 50, cv2.BORDER_CONSTANT, value=[255])

    # Up-sample (optional)
    img = cv2.resize(img, (0, 0), fx=2, fy=2)

    # Convert to gray-scale
    gry = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to create a binary image
    _, binary_img = cv2.threshold(gry, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    kernel = np.ones((5,5),np.uint8)
    erosion = cv2.erode(img, kernel, iterations = 1)

    # OCR
    txt = pt.image_to_string(erosion, config="--oem 3 --psm 6")

    print(txt)
else:
    print("Failed to load the image.")

show_image(path)

In [ ]:
import sys
import numpy as np
from cv2 import *
import time
import os
import threading
import pytesseract
import imutils

In [ ]:
def get_string(img_path):
    # Read image using opencv
    img = cv2.imread(img_path)
   # Extract the file name without the file extension
    file_name = os.path.basename(img_path).split('.')[0]
    file_name = file_name.split()[0]
    # Create a directory for outputs
    output_path = os.path.join('output_path', "ocr")
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    # Rescale the image, if needed.
    img = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
    # Converting to gray scale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #Removing Shadows
    rgb_planes = cv2.split(img)
    result_planes = []
    result_norm_planes = []
    for plane in rgb_planes:
        dilated_img = cv2.dilate(plane, np.ones((7,7), np.uint8))
        bg_img = cv2.medianBlur(dilated_img, 21)
        diff_img = 255 - cv2.absdiff(plane, bg_img)
        result_planes.append(diff_img)
    img = cv2.merge(result_planes)
    
    #Apply dilation and erosion to remove some noise
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)#increases the white region in the image 
    img = cv2.erode(img, kernel, iterations=1) #erodes away the boundaries of foreground object
    
    #Apply blur to smooth out the edges
    #img = cv2.GaussianBlur(img, (5, 5), 0)
    
    # Apply threshold to get image with only b&w (binarization)
    img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    
    #Save the filtered image in the output directory
    save_path = os.path.join(output_path, file_name + "_filter_" + str('as') + ".png")
    cv2.imwrite(save_path, img)
    # Recognize text with tesseract for python
    result = pytesseract.image_to_string(img, lang="eng")
    return result

In [ ]:
s = get_string(path)
t = s.split(sep='\n')
for s in t:
    print(s)

In [ ]:
# Check Image Quality 
import cv2

def check_image_quality(image_path):
    # Load the image
    img = cv2.imread(image_path)

    # Check resolution
    height, width = img.shape[:2]
    #print height, width
    print("Image resolution: {} x {}".format(height, width))

    min_resolution = (700, 700)  # Set a minimum resolution threshold
    if height < min_resolution[0] or width < min_resolution[1]:
        print("Image resolution is too low for OCR")


# Example usage
print(path)
check_image_quality(path)
